<h1><center>TP4 - Ex.3</center></h1>
<p><center>May 27, 2024</center></p>


### Estruturas Criptográficas

PG53886, Ivo Miguel Alves Ribeiro

A95323, Henrique Ribeiro Fernandes


3. Construir tabelas de comparações das suas implementações, para os vários níveis de segurança  NIST e em termos dos seguintes parâmetros
        1. Tempos: geração das chaves, produção da assinatura e verificação da assinatura.
        2. Tamanhos: da chave pública, da chave privada e da assinatura.

In [1]:
import time
import pandas as pd
from pmain.dilithium import Dilithium2, Dilithium3, Dilithium5
from sphincs import sphincs

# Função para medir o tempo de execução
def measure_time(func, *args):
    start_time = time.time()
    result = func(*args)
    end_time = time.time()
    return result, end_time - start_time

# Testando diferentes níveis de segurança
security_levels = ['ML-DSA-44', 'ML-DSA-65', 'ML-DSA-87']
results = []

for level in security_levels:
    # Configure as variáveis específicas para cada nível de segurança
    if level == 'ML-DSA-44':
        dilithium = Dilithium2
    elif level ==  'ML-DSA-65':
        dilithium = Dilithium3
    elif level == 'ML-DSA-87':
        dilithium = Dilithium5

    # Medir tempos e tamanhos
    (pk, sk), keygen_time = measure_time(dilithium.keygen)
    msg = b"Your message signed by Dilithium"
    sig, sign_time = measure_time(dilithium.sign, sk, msg)
    verify_time = measure_time(dilithium.verify, pk, msg, sig)[1]

    public_key_size = len(pk)
    private_key_size = len(sk)
    signature_size = len(sig)

    results.append({
        'Name': 'Dilithium'+level,
        'Keygen Time (s)': keygen_time,
        'Sign Time (s)': sign_time,
        'Verify Time (s)': verify_time,
        'Public Key Size (bytes)': public_key_size,
        'Private Key Size (bytes)': private_key_size,
        'Signature Size (bytes)': signature_size
    })

# Medir tempos e tamanhos
(sk, pk), keygen_time = measure_time(sphincs.spx_keygen)
msg = b"Your message signed by Dilithium"
sig, sign_time = measure_time(sphincs.spx_sign, msg, sk)
verify_time = measure_time(sphincs.spx_verify, msg, sig, pk)[1]

public_key_size = len(pk)
private_key_size = len(sk)
signature_size = len(sig)

results.append({
    'Name':'Sphincs SLH-DSA-SHAKE-256s',
    'Keygen Time (s)': keygen_time,
    'Sign Time (s)': sign_time,
    'Verify Time (s)': verify_time,
    'Public Key Size (bytes)': public_key_size,
    'Private Key Size (bytes)': private_key_size,
    'Signature Size (bytes)': signature_size
})

# Criar um DataFrame com os resultados
df = pd.DataFrame(results)

# Exibir a tabela de resultados
print(df)


                         Name  Keygen Time (s)  Sign Time (s)  \
0          DilithiumML-DSA-44         0.014241       0.056340   
1          DilithiumML-DSA-65         0.019064       0.135381   
2          DilithiumML-DSA-87         0.025388       0.075831   
3  Sphincs SLH-DSA-SHAKE-256s         0.841802      13.272503   

   Verify Time (s)  Public Key Size (bytes)  Private Key Size (bytes)  \
0         0.014105                     1312                      2528   
1         0.021570                     1952                      4000   
2         0.030301                     2592                      4864   
3         0.009735                       64                       128   

   Signature Size (bytes)  
0                    2420  
1                    3293  
2                    4595  
3                   29792  
